In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')

In [3]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')

In [4]:
data = pd.concat([train, test], ignore_index=True, sort=False)

In [5]:
data.reset_index(drop=True, inplace=True)

In [6]:
data.shape

(31015, 4)

In [7]:
def prepare(text):
    words = text.split()
    retval, first_char, invert_map = [], [], []
    current_pos = 0
    for w in words:
        word_ret = [""]
        word_invert = [current_pos]
        for p, c in enumerate(w):
            if c in ['.',',','!','?']:
                if word_ret[-1]=="":
                    word_ret[-1]+=c
                    word_invert[-1]=current_pos+p
                else:
                    word_ret.append(c)
                    word_invert.append(current_pos+p)
                word_ret.append("")
                word_invert.append(current_pos+p+1)
            else:
                word_ret[-1]+=c
        if len(word_ret[-1])==0:
            word_ret.pop(-1)
            word_invert.pop(-1)
        word_first = [False if i>0 else True for i in range(len(word_ret)) ]
        retval.extend(word_ret)
        first_char.extend(word_first)
        invert_map.extend(word_invert)
        current_pos+=len(w)+1
    assert len(retval)==len(first_char)
    return retval, first_char, invert_map

In [8]:
data.tail()

,textID,text,selected_text,sentiment
31010,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",NaN,negative
31011,416863ce47,All alone in this old house again. Thanks for...,NaN,positive
31012,6332da480c,I know what you mean. My little dog is sinkin...,NaN,negative
31013,df1baec676,_sutra what is your next youtube video gonna b...,NaN,positive
31014,469e15c5a8,http://twitpic.com/4woj2 - omgssh ang cute n...,NaN,positive


In [9]:
data.dropna(subset=['text'], how='any', inplace=True)

In [10]:
data['text'] = data['text'].apply(lambda x: ' '.join(x.lower().strip().split()))

In [11]:
data['text'].tail()

31010    its at 3 am, im very tired but i can`t sleep b...
31011    all alone in this old house again. thanks for ...
31012    i know what you mean. my little dog is sinking...
31013    _sutra what is your next youtube video gonna b...
31014    http://twitpic.com/4woj2 - omgssh ang cute ng ...
Name: text, dtype: object

In [12]:
data['text'] = data['text'].apply(lambda st: st.replace("`","'"))

In [13]:
with open('../input/corpus_aug.txt', 'w') as f:
    for sentiment, text in zip(data['sentiment'].tolist(), data['text'].tolist()):
        words, _, _ = prepare(text)
        text = ' '.join(words)
        f.write(text+"\n")